<a href="https://colab.research.google.com/github/ShikharV010/gist_daily_runs/blob/main/Backlinks_data_from_vendor_Manufacturing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

sheet_id = "1wpDlN402yZshsQRERU1Ki1xWQGpMieSO7GnxeQlbOAE"  # from your URL
gid = "0"                                                # tab gid from the URL
csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"

df = pd.read_csv(csv_url)  # <- done
df.head()


,Sr no.,Date Submitted,Custom (Yes/No),Custom (Sheet for custom websites),Client Name,Month,Keywords,Search Volume,Keyword Difficulty,Ranking URL,Location,Position,Type,Anchor Text Requested,Article Topic,Doc URL,Live URL,Vendor Name,Link live Date,Billing Date
0,1,25th August 2025,No,NaN,johnmayecompany.com,August 2025,John Maye Company,NaN,NaN,https://johnmayecompany.com/,US,NaN,HQL,John Maye Company,How Semi Automatic Stretch Wrappers Improve Lo...,How Semi-Automatic Stretch Wrappers Improve Lo...,Not Live,NaN,NaN,NaN
1,2,25th August 2025,No,NaN,johnmayecompany.com,August 2025,John Maye Company,NaN,NaN,https://johnmayecompany.com/,US,NaN,HQL,https://johnmayecompany.com/,The Role of Banding Machines in Reducing Packa...,The Role of Banding Machines in Reducing Packa...,Not Live,NaN,NaN,NaN
2,3,25th August 2025,No,NaN,johnmayecompany.com,August 2025,semi automatic stretch wrapper,NaN,NaN,https://johnmayecompany.com/pallet-wrapping-eq...,US,NaN,HQL,best semi automatic stretch wrapper,Why Semi Automatic Strapping Machines Are Idea...,Why Semi Automatic Strapping Machines Are Idea...,Not Live,NaN,NaN,NaN
3,4,25th August 2025,No,NaN,johnmayecompany.com,August 2025,banding machine,NaN,NaN,https://johnmayecompany.com/strapping-and-band...,US,NaN,HQL,banding machines,Balancing Cost and Productivity: Choosing Betw...,Balancing Cost and Productivity: Choosing Betw...,Not Live,NaN,NaN,NaN
4,5,25th August 2025,No,NaN,johnmayecompany.com,August 2025,semi automatic strapping machines,NaN,NaN,https://johnmayecompany.com/strapping-and-band...,US,NaN,HQL,best semi automatic strapping machines,Banding vs. Strapping: Understanding the Right...,Banding vs. Strapping: Understanding the Right...,Not Live,NaN,NaN,NaN


In [6]:
# --- Install deps (once per runtime) ---
!pip -q install SQLAlchemy psycopg2-binary
import os
import pandas as pd
from sqlalchemy import create_engine, inspect, text
from datetime import datetime, timezone
import hashlib

# ─────────── 1) READ GOOGLE SHEET (Option A via CSV export) ───────────
sheet_id = "1wpDlN402yZshsQRERU1Ki1xWQGpMieSO7GnxeQlbOAE"   # from your URL
gid      = "0"                                             # tab gid from the URL
csv_url  = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"

df = pd.read_csv(csv_url)                                   # <- your data

# ─────────── 2) DB CONFIG (RDS URL you gave) ───────────
DB_URL = os.getenv("POSTGRES_DB_URL")
assert DB_URL, "Missing POSTGRES_DB_URL env var."
engine = create_engine(DB_URL)
TABLE_SCHEMA = "gist"             # ← change if needed
TABLE_NAME   = "backlinks_vendor_data"     # ← change if needed

engine = create_engine(DB_URL)
insp   = inspect(engine)

# ─────────── 3) NORMALIZE + BUILD DEDUP HASH ───────────
df_norm = df.copy()
# Postgres-friendly column names
df_norm.columns = [c.strip().lower().replace(" ", "_") for c in df_norm.columns]

# Stable hash across all columns (treat NaN as "")
df_for_hash = df_norm.fillna("").astype(str)
row_hashes = (
    df_for_hash.apply(lambda r: "|".join(r.values.tolist()), axis=1)
               .map(lambda s: hashlib.sha1(s.encode("utf-8")).hexdigest())
)
df_norm["row_hash"] = row_hashes

# ─────────── 4) LOAD EXISTING HASHES (if table already exists) ───────────
table_exists = insp.has_table(TABLE_NAME, schema=TABLE_SCHEMA)
existing_hashes = set()

with engine.begin() as conn:
    if table_exists:
        existing_df = pd.read_sql(
            text(f'SELECT row_hash FROM "{TABLE_SCHEMA}"."{TABLE_NAME}"'),
            conn
        )
        existing_hashes = set(existing_df["row_hash"].tolist())

# Keep only new rows
new_rows = df_norm[~df_norm["row_hash"].isin(existing_hashes)].copy()

if new_rows.empty:
    print("No new rows to insert (all rows already ingested).")
else:
    # Add ingestion timestamp (same moment for this batch)
    new_rows["ingested_at"] = datetime.now(timezone.utc)

    # ─────────── 5) APPEND ───────────
    new_rows.to_sql(
        TABLE_NAME,
        engine,
        schema=TABLE_SCHEMA,
        if_exists="append",    # creates the table if it doesn't exist
        index=False,
        method="multi",
        chunksize=1000,
    )

    # ─────────── 6) SAFETY: ENSURE UNIQUE INDEX ON row_hash ───────────
    with engine.begin() as conn:
        conn.execute(text(
            f'CREATE UNIQUE INDEX IF NOT EXISTS ux_{TABLE_NAME}_row_hash '
            f'ON "{TABLE_SCHEMA}"."{TABLE_NAME}" (row_hash);'
        ))

    print(f"Inserted {len(new_rows)} new rows into {TABLE_SCHEMA}.{TABLE_NAME}.")


AssertionError: Missing POSTGRES_DB_URL env var.